In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import altair as alt

In [4]:
df = pd.read_csv('..\data\imputed_dataset.csv')
df.head()

,rating,ranking,year,position,height,weight,latitude,longitude,state_province,stars,committed_to,is_drafted,conference,latitude_school,longitude_school,distance_miles,side_of_ball,position_group
0,0.9992,1.0,2015,DT,74.0,313.0,31.578206,-84.155681,GA,5,Georgia,0.0,SEC,33.94982,-83.373381,170.049603,defense,d_line
1,0.9991,1.0,2015,DT,74.5,313.0,31.578206,-84.155681,GA,5,Georgia,0.0,SEC,33.94982,-83.373381,170.049603,defense,d_line
2,0.9879,27.0,2015,ATH,72.0,168.0,33.173177,-84.914936,GA,5,Georgia,1.0,SEC,33.94982,-83.373381,103.714733,athlete,athlete
3,0.9769,48.0,2015,OLB,74.0,207.0,32.305158,-84.027407,GA,4,Georgia,1.0,SEC,33.94982,-83.373381,119.771399,defense,linebacker
4,0.9680,65.0,2015,SDE,76.0,265.0,33.853270,-84.220073,GA,4,Georgia,0.0,SEC,33.94982,-83.373381,49.011958,defense,d_line


In [5]:
# Define numerical and categorical values
numerical_features = ['rating', 'ranking', 'height', 'weight', 'distance_miles', 'stars']
categorical_features = ['side_of_ball', 'position_group']
target = ['is_drafted']


In [10]:
# Features and target
X = df[numerical_features]
y = df[target]

In [11]:
model = DBSCAN()
clusters = model.fit_predict(X)

In [12]:
df['Cluster'] = clusters

array([-1, -1, -1, ..., -1, -1, -1])